# Contraceptive Method Choice
Lab Assignment Four: Extending Logistic Regression

**_Jake Oien, Seung Ki Lee, Jenn Le_**

## Data Preparation and Overview

### Business Case

### Class Variables

Dataset Source: https://archive.ics.uci.edu/ml/datasets/Contraceptive+Method+Choice

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Here, we'll import the data and add column names
col_names = ["Age", 
             "Wife's Education", 
             "Husband's Education", 
             "# of Children", 
             "Wife's Religion", 
             "Wife Working", 
             "Husband's Occupation",
             "SoL Index",
             "Media Exposure",
             "Contraceptive Method"
            ]
data = pd.read_csv("./cmc.csv", header=None, names=col_names, encoding='latin-1')

data.head()

,Age,Wife's Education,Husband's Education,# of Children,Wife's Religion,Wife Working,Husband's Occupation,SoL Index,Media Exposure,Contraceptive Method
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1


### Data Division

## Modeling

### Custom Logistic Regression Classifier
_ # Make your implementation of logistic regression compatible with the GridSearchCV function that is part of scikit-learn _

### Training

### Comparison